In [2]:
from lavis.models import load_model_and_preprocess
import torch
import os
from PIL import Image
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import cv2
import pandas as pd

torch_dtype = torch.float32
# torch_dtype = torch.float16
# load the BLIP model
model_blip, vis_processors, _ = load_model_and_preprocess(name="blip_caption", model_type="base_coco", is_eval=True, device=torch.device("cuda"))

In [3]:
data_path = '/grogu/user/akirchme/art3d_sd_webui'
cls = 'dog'
from tqdm import tqdm 

files = sorted([f for f in listdir(f'{data_path}/fg/{cls}') if isfile(join(f'{data_path}/fg/{cls}', f))])

output_dir = '/grogu/user/akirchme/art3d_sd_webui/prompt'
os.makedirs(f'{output_dir}/fg/{cls}', exist_ok=True)
os.makedirs(f'{output_dir}/images/{cls}', exist_ok=True)

for file in tqdm(files[::-1], desc=cls):
    file_id = file.split('.')[0]
    for folder in ['images', 'fg']:
        out_file = f'{output_dir}/{folder}/{cls}/{file_id}.txt'
        if os.path.exists(out_file):
            continue
        img_path = f'{data_path}/{folder}/{cls}/{file_id}.png'
        img = Image.open(img_path).resize((512,512), Image.Resampling.LANCZOS)
        _image = vis_processors["eval"](img).unsqueeze(0).cuda()
        prompt_str = model_blip.generate({"image": _image})[0]
        with open(out_file, "w") as f:
            f.write(prompt_str)

dog: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [05:06<00:00,  6.53it/s]


In [5]:
open(f'{output_dir}/{folder}/{cls}/0.txt').read()

"a close up of a dog's face with blue eyes"